# Extract from Wikipedia Dump

- Download from https://dumps.wikimedia.org/kawiki/latest/kawiki-latest-pages-articles-multistream.xml.bz2
- Unzip with `bzip2 -dk embeddings/data/raw/kawiki-20220520-pages-articles-multistream.xml.bz2`
- Proceed with BS4


> Alternatively use WikiExtractor which doesn't produce clean results. 1/4 of articles were empty. Example: https://ka.wikipedia.org/wiki?curid=610

In [1]:
import pandas as pd
# import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
import wikitextparser as wtp
from tqdm import tqdm
from pandarallel import pandarallel

pandarallel.initialize(progress_bar=True, nb_workers=6)


INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
dump = open("../raw/kawiki-20220520-pages-articles-multistream.xml","r")
contents = dump.read()
soup = BeautifulSoup(contents,'xml')

In [3]:
pages = soup.find_all('page')
print(len(pages))

320745


In [11]:
pages[1500].find('ns').text == '0'

True

In [12]:
kawiki = []
for page in tqdm(pages):
    # Extract only the mainspace/article namespace
    if page.find("ns").text != '0':
        continue
    
    wiki = {
        
        'id' : page.find('id').text,
        'ts' : page.find('timestamp').text,
        'title': page.find('title').text,
        'author': page.find('contributor').find('username').text if page.find('contributor').find('username') != None else page.find('contributor').find('ip').text,
        'text': page.find('text').text,
        'sha1': page.find('sha1').text,
    }

    kawiki.append(wiki)

kawiki = pd.DataFrame(kawiki)

100%|██████████| 320745/320745 [00:31<00:00, 10115.94it/s]


In [13]:
kawiki

,id,ts,title,author,text,sha1
0,604,2022-04-15T07:50:59Z,ედუარდ შევარდნაძე,Melberg,{{ინფოდაფა ლიდერი\n| სახელი = ედუარდ შევარდნაძ...,k4tckhizmxggllhid4m0uob0nmwbu0h
1,608,2020-12-01T07:15:40Z,1 იანვარი,Namdvili,{{კალ|{{CURRENTYEAR}}|1}}\n'''1 იანვარი''' — [...,9hua4ulox6vj429j2nbx3404fylhw5t
2,609,2020-01-18T07:59:48Z,25 იანვარი,95.104.15.233,{{კალ|{{CURRENTYEAR}}|1}}\n'''25 იანვარი''' — ...,nburza2tu9lqypz9cirawwpcg3s7e8z
3,610,2021-11-08T19:32:05Z,1928,Arkaitz1974,{{წელი|{{PAGENAME}}}}\n== მოვლენები ==\n* [[11...,g2uj2d76l6yrbtu1p4wvmnx7vacz64z
4,823,2022-05-09T20:01:56Z,საქართველო,GiorgiXIII,{{მმ}}\n{{ინფოდაფა ქვეყანა2\n|ვირტუალური სახელ...,9kmab2upb1nr0ad1ywgr6ldl787kyo3
...,...,...,...,...,...,...
207439,532938,2022-05-20T04:35:44Z,ვახტანგ რობაქიძე,Jaba1977,ვახტანგ რობაქიძე (მოსამართლეობის კანდიდატი) და...,iavvdscaounfaz6sqowhc2ogsvp4u0l
207440,532939,2022-05-20T08:54:31Z,ნინა კარპაჩოვა,Ekkatterrinna,{{ინფოდაფა პიროვნება|name_color=white|back_col...,m955153juz99zg6n3e4pphtna8s6f8t
207441,532941,2022-05-19T23:20:41Z,Los Williames,185.225.28.38,[[File:Risitas.jpg|220px]],dagmtfb81asi3gnehcjfw5l1fhj4f0i
207442,532947,2022-05-20T08:27:24Z,ჰაილბრონის აჩრდილი,Lisztomaniac,{{მოკლე აღწერა|ნავარაუდევი სერიული მკვლელი ქალ...,6sgtfdyggb9svx8whe22owjhwkm8fia


In [14]:
kawiki['text'] = kawiki['text'].parallel_apply(lambda row: wtp.remove_markup(row))
kawiki['isRedirection'] = kawiki.text.str.startswith('#გადამისამართება') | kawiki.text.str.startswith('#REDIRECT')

In [15]:
kawiki.isRedirection.sum()

46423

In [16]:
kawiki

,id,ts,title,author,text,sha1,isRedirection
0,604,2022-04-15T07:50:59Z,ედუარდ შევარდნაძე,Melberg,\nედუარდ ამბროსის ძე შევარდნაძე (დ. 25 იანვარი...,k4tckhizmxggllhid4m0uob0nmwbu0h,False
1,608,2020-12-01T07:15:40Z,1 იანვარი,Namdvili,\n1 იანვარი — გრიგორიანული კალენდრის პირველი დ...,9hua4ulox6vj429j2nbx3404fylhw5t,False
2,609,2020-01-18T07:59:48Z,25 იანვარი,95.104.15.233,\n25 იანვარი — გრიგორიანული კალენდრის 25-ე დღე...,nburza2tu9lqypz9cirawwpcg3s7e8z,False
3,610,2021-11-08T19:32:05Z,1928,Arkaitz1974,\n== მოვლენები ==\n* 11 თებერვალი : შვეიცარიის...,g2uj2d76l6yrbtu1p4wvmnx7vacz64z,False
4,823,2022-05-09T20:01:56Z,საქართველო,GiorgiXIII,"\n\nსაქართველო () — სახელმწიფო ევრაზიაში, კავკ...",9kmab2upb1nr0ad1ywgr6ldl787kyo3,False
...,...,...,...,...,...,...,...
207439,532938,2022-05-20T04:35:44Z,ვახტანგ რობაქიძე,Jaba1977,ვახტანგ რობაქიძე (მოსამართლეობის კანდიდატი) და...,iavvdscaounfaz6sqowhc2ogsvp4u0l,False
207440,532939,2022-05-20T08:54:31Z,ნინა კარპაჩოვა,Ekkatterrinna,\n\nნინა კარპაჩოვა (უკრ. Ні́на Іва́нівна Карпа...,m955153juz99zg6n3e4pphtna8s6f8t,False
207441,532941,2022-05-19T23:20:41Z,Los Williames,185.225.28.38,220px,dagmtfb81asi3gnehcjfw5l1fhj4f0i,False
207442,532947,2022-05-20T08:27:24Z,ჰაილბრონის აჩრდილი,Lisztomaniac,\nthumb|right|250px|მემორიალური ნიშნული ოფიცერ...,6sgtfdyggb9svx8whe22owjhwkm8fia,False


In [13]:
kawiki.to_csv("../raw/kawiki.csv", index=None, encoding='utf-8')